In [ ]:
count=int(input('크롤링한 블로그 갯수 : '))

크롤링한 블로그 갯수 : 1000


In [ ]:
# !sudo apt-get install curl
# !pip install pyLDAvis
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import argparse
import torch
import collections
import os
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
from gensim import corpora
from tqdm import tqdm

In [ ]:
!pip install pandas --upgrade

In [ ]:
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['font.size'] = 12

Mecab 및 Konlpy 설치

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!set -x/
!pip install konlpy

전처리 & 말뭉치 파일 생성

In [ ]:
from konlpy.tag import Mecab
import re

mecab = Mecab()

# 크롤링 csv 파일 불러오기
여행 = pd.read_csv(f"/content/drive/Shareddrives/과제 데이터/전남 화순 sns 분석을 통한 관광지 홍보/Data^^/화순여행_크롤링.csv")
trip_text = 여행['Text']

# series에서 string으로 변환
trip_str = '\n'.join(str(trip_text[i]) for i in range(len(trip_text)))

# 이모지 제거
only_BMP_pattern = re.compile("["
                              u"\U00010000-\U0010FFFF" # BMP characters 이외
                              "]+", flags=re.UNICODE)
trip_str = only_BMP_pattern.sub(r'', trip_str) # BMP characters만

# 특수문자 & 한자 & 영어 제거
trip_str = re.sub('[^0-9가-힣\n]', ' ', trip_str)

# 단일 자음 모음 제거
trip_str = re.sub('[ㄱ-ㅎㅏ-ㅣ]', ' ', trip_str)

# 연속 공백 하나로 합치기
trip_str = re.sub(' +', ' ', trip_str)

# 말뭉치 string을 텍스트 파일로 저장
trip_file = '/content/drive/Shareddrives/과제 데이터/전남 화순 sns 분석을 통한 관광지 홍보/Data^^/trip_file.txt'
with open(trip_file, 'w', encoding='utf-8') as f:
    f.write(trip_str)

In [ ]:
trip_file

'/content/drive/Shareddrives/과제 데이터/전남 화순 sns 분석을 통한 관광지 홍보/Data^^/trip_file.txt'

환경설정

In [ ]:
args = {
    "seed": 1234,
    # CPU 또는 GPU 사용 여부 선택
    # 코랩 GPU 적용되었는지 확인할 것!
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    # 말뭉치 파일
    "corpus": trip_file,
}
args = argparse.Namespace(**args)
print(args)

Namespace(corpus='/content/drive/Shareddrives/과제 데이터/전남 화순 sns 분석을 통한 관광지 홍보/Data^^/trip_file.txt', device=device(type='cuda'), seed=1234)


token 별 빈도 딕셔너리 생성 (명사)

In [ ]:
morph_counter = collections.defaultdict(int) # 딕셔너리 생성
morph_nouns = []

# 불용어 사전 불러오기
stop_words=open('/content/drive/Shareddrives/과제 데이터/전남 화순 sns 분석을 통한 관광지 홍보/Data^^/불용어사전1.txt')
stop_W = stop_words.read().split('\n')

with open(args.corpus) as f: # 말뭉치 텍스트 파일을 열고
    for i, line in enumerate(f): # 각 인덱스와 줄에 대하여
        tmp = []
        for m in mecab.nouns(line): # 명사만 추출하고
            if len(m) > 1 and m not in stop_W: # 2글자 이상이면서 불용어에 없는
                morph_counter[m] += 1 # 형태소의 개수를 센다.
                tmp.append(m)
        if len(tmp) != 0:
            morph_nouns.append(tmp) # 각 블로그 중에서 명사의 길이가 2 이상인 것들을 리스트에 모은다.

In [ ]:
# 형태소(명사)의 중복을 허용하지 않는 개수
print(len(morph_counter))

15810


In [ ]:
# 빈도수가 많은 형태소 10개 출력
most_freq = sorted(morph_counter.items(), key=lambda item: item[1], reverse=True)
print(most_freq[:10])

[('고인돌', 1608), ('운주사', 1387), ('마을', 1154), ('시간', 1130), ('공원', 1045), ('사람', 1005), ('이곳', 962), ('생각', 857), ('코스', 849), ('우리', 835)]


In [ ]:
# 빈도수가 적은 형태소 10개 출력
most_freq = sorted(morph_counter.items(), key=lambda item: item[1], reverse=False)
print(most_freq[:10])

[('열풍', 1), ('흥청', 1), ('올리고당', 1), ('크래커', 1), ('냉난', 1), ('캠린', 1), ('도청', 1), ('장남', 1), ('내추럴', 1), ('남상', 1)]


In [ ]:
# token의 중복을 허용하는 총 개수
count = 0
for c, cnt in morph_counter.items(): # 형태소와 각 형태소에 대한 개수
    count += cnt # 개수를 다 더해서 전체 토큰의 수를 구함
print(count)

187443


In [ ]:
# morpheme(형태소)에 일련번호(int) 부여
morph_to_id = {'[PAD]': 0, '[UNK]': 1} # 문장의 길이를 맞추기 위한 '[PAD]', 없는 글자를 처리하기 위한 '[UNK]'
for w, cnt in morph_counter.items():
    morph_to_id[w] = len(morph_to_id)
    
print(len(morph_to_id))
print(morph_to_id)

LDA(Latent Dirichlet Allocation) 잠재 디리클레 할당



In [ ]:
dictionary = corpora.Dictionary(morph_nouns)
corpus = [dictionary.doc2bow(text) for text in morph_nouns]
print(corpus)

In [ ]:
print(len(dictionary))
print(dictionary[0], dictionary[1], dictionary[2], dictionary[3], dictionary[4])

15810
가능 가족 객실 거리 게스트


In [ ]:
NUM_TOPICS = 3 # K개의 토픽
ldamodel = gensim.models.ldamulticore.LdaMulticore(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15, workers=os.cpu_count())
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

In [ ]:
print(ldamodel.print_topics())

각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여준다. 또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 5개의 토픽은 0부터 4까지의 번호가 할당되어져 있다. 아래 그래프에서 원(토픽)이 서로 겹치지 않으면서 가장 큰 수를 적용하였다. passes는 알고리즘의 동작 횟수를 말하는데, 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 된다. 여기서는 총 15회를 수행했다. 여기서는 num_words=5로 총 5개의 단어만 출력하도록 하였다.

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.032089 -0.088302       1        1  40.762198
2     -0.126573  0.024924       2        1  37.650446
1      0.094483  0.063378       3        1  21.587356, topic_info=     Term         Freq        Total Category  logprob  loglift
1457  운주사  1296.000000  1296.000000  Default  30.0000  30.0000
342   고인돌  1506.000000  1506.000000  Default  29.0000  29.0000
1441   석탑   538.000000   538.000000  Default  28.0000  28.0000
166    국화   740.000000   740.000000  Default  27.0000  27.0000
1770   불상   412.000000   412.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
31     다음   102.367893   369.259122   Topic3  -5.9796   0.2501
110    정도   115.490440   640.927233   Topic3  -5.8590  -0.1807
415    이곳   116.628518   893.322087   Topic3  -5.8492  -0.5029
95     이번    97.241175   344.332341   Topic3  -6.0310   0.2686
98     이용    96.898976   403.359813   Topic3  -6.0345   0.1069

[264 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
15013      1  0.925945    가연
148        1  0.363373    가을
148        2  0.502358    가을
148        3  0.135637    가을
1          1  0.510479    가족
...      ...       ...   ...
1166       2  0.933180  화순적벽
1166       3  0.014033  화순적벽
333        1  0.664105    힐링
333        2  0.186369    힐링
333        3  0.149620    힐링

[497 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

문서 별 토픽 분포

In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [ ]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:15]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,0.0,0.8642,"[(0, 0.86415774), (1, 0.13420852)]"
1,1,0.0,0.8184,"[(0, 0.81841457), (1, 0.18051423)]"
2,2,0.0,0.9979,"[(0, 0.9979093)]"
3,3,0.0,0.9942,"[(0, 0.994219)]"
4,4,1.0,0.9975,"[(1, 0.9974962)]"
5,5,0.0,0.9961,"[(0, 0.9961294)]"
6,6,2.0,0.9931,"[(2, 0.9931391)]"
7,7,0.0,0.7541,"[(0, 0.7541446), (1, 0.08277724), (2, 0.163078..."
8,8,1.0,0.6992,"[(0, 0.2998487), (1, 0.6991919)]"
9,9,2.0,0.9439,"[(1, 0.05442495), (2, 0.943878)]"


In [ ]:
topictable.groupby('가장 비중이 높은 토픽').count()

,문서 번호,가장 높은 토픽의 비중,각 토픽의 비중
가장 비중이 높은 토픽,,,
0.0,416,416,416
1.0,283,283,283
2.0,258,258,258
